# インポート

In [1]:
import pandas as pd
import pickle

import numpy as np

import scraping
import create_rawdf
import create_prediction_population
from pathlib import Path
import requests

import scraping_prediction

DATA_DIR = Path("..", "data")
HTML_RACE_DIR = DATA_DIR / "html" / "race"
HTML_HORSE_DIR = DATA_DIR / "html" / "horse"
HTML_PED_DIR = DATA_DIR / "html" / "ped"
HTML_LEADING_DIR = DATA_DIR / "html" / "leading"
%load_ext autoreload

In [2]:
# モジュールの変更を反映させたい時実行
%autoreload

# データ取得

In [70]:
#全部載せ
# 一時保存ディレクトリ
TMP_DIR = scraping.DATA_DIR / "tmp_new"

In [71]:
# 開催日一覧の取得
kaisai_date_list = scraping.scrape_kaisai_date(
    from_="2025-02", to_="2025-02", save_dir=TMP_DIR
)

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.21s/it]


In [38]:
# 途中まで処理したlistからスクレイピングするための前処理
def load_kaisai_date_list_2(file_path: Path) -> list[str]:
    with open(file_path, "r") as f:
        return f.read().splitlines()

kaisai_date_file = Path(TMP_DIR / "kaisai_date_list_2.txt")
kaisai_date_list_2 = load_kaisai_date_list_2(kaisai_date_file)
#全て終わった場合、kaisai_date_listにkaisai_date_list_2の内容をくっつけること

In [47]:
kaisai_date_list

['20250201',
 '20250202',
 '20250208',
 '20250209',
 '20250210',
 '20250215',
 '20250216',
 '20250222',
 '20250223']

In [72]:
# スクレイピング対象レースのid取得
race_id_list = scraping.scrape_race_id_list(
    kaisai_date_list, save_dir=TMP_DIR
)

100%|█████████████████████████████████████████████| 9/9 [01:36<00:00, 10.67s/it]


In [127]:
# notebookの再起動などでrace_id_listが失われた場合はファイルから読み込む
with open(TMP_DIR / "race_id_list.txt", "r") as f:
    race_id_list = f.read().splitlines()

In [128]:
len(race_id_list)

31346

In [45]:
race_id_list

['201606010101',
 '201606010102',
 '201606010103',
 '201606010104',
 '201606010105',
 '201606010106',
 '201606010107',
 '201606010108',
 '201606010109',
 '201606010110',
 '201606010111',
 '201606010112',
 '201608010101',
 '201608010102',
 '201608010103',
 '201608010104',
 '201608010105',
 '201608010106',
 '201608010107',
 '201608010108',
 '201608010109',
 '201608010110',
 '201608010111',
 '201608010112',
 '201606010201',
 '201606010202',
 '201606010203',
 '201606010204',
 '201606010205',
 '201606010206',
 '201606010207',
 '201606010208',
 '201606010209',
 '201606010210',
 '201606010211',
 '201606010212',
 '201608010201',
 '201608010202',
 '201608010203',
 '201608010204',
 '201608010205',
 '201608010206',
 '201608010207',
 '201608010208',
 '201608010209',
 '201608010210',
 '201608010211',
 '201608010212',
 '201606010301',
 '201606010302',
 '201606010303',
 '201606010304',
 '201606010305',
 '201606010306',
 '201606010307',
 '201606010308',
 '201606010309',
 '201606010310',
 '201606010311

In [66]:
#すでにあるレースを手動で取ってくる方法
race_id_list2 = """202506010101
202506010102
202506010103
202506010104
202506010105
202506010106
202506010107
202506010108
202506010109
202506010110
202506010111
202506010112
202507010101
202507010102
202507010103
202507010104
202507010105
202507010106
202507010107
202507010108
202507010109
202507010110
202507010111
202507010112
202506010201
202506010202
202506010203
202506010204
202506010205
202506010206
202506010207
202506010208
202506010209
202506010210
202506010211
202506010212
202507010201
202507010202
202507010203
202507010204
202507010205
202507010206
202507010207
202507010208
202507010209
202507010210
202507010211
202507010212
202506010301
202506010302
202506010303
202506010304
202506010305
202506010306
202506010307
202506010308
202506010309
202506010310
202506010311
202506010312
202507010301
202507010302
202507010303
202507010304
202507010305
202507010306
202507010307
202507010308
202507010309
202507010310
202507010311
202507010312
202506010401
202506010402
202506010403
202506010404
202506010405
202506010406
202506010407
202506010408
202506010409
202506010410
202506010411
202506010412
202507010401
202507010402
202507010403
202507010404
202507010405
202507010406
202507010407
202507010408
202507010409
202507010410
202507010411
202507010412
202506010501
202506010502
202506010503
202506010504
202506010505
202506010506
202506010507
202506010508
202506010509
202506010510
202506010511
202506010512
202507010501
202507010502
202507010503
202507010504
202507010505
202507010506
202507010507
202507010508
202507010509
202507010510
202507010511
202507010512
202506010601
202506010602
202506010603
202506010604
202506010605
202506010606
202506010607
202506010608
202506010609
202506010610
202506010611
202506010612
202507010601
202507010602
202507010603
202507010604
202507010605
202507010606
202507010607
202507010608
202507010609
202507010610
202507010611
202507010612
202506010701
202506010702
202506010703
202506010704
202506010705
202506010706
202506010707
202506010708
202506010709
202506010710
202506010711
202506010712
202507010701
202507010702
202507010703
202507010704
202507010705
202507010706
202507010707
202507010708
202507010709
202507010710
202507010711
202507010712
202506010801
202506010802
202506010803
202506010804
202506010805
202506010806
202506010807
202506010808
202506010809
202506010810
202506010811
202506010812
202507010801
202507010802
202507010803
202507010804
202507010805
202507010806
202507010807
202507010808
202507010809
202507010810
202507010811
202507010812
202510010101
202510010102
202510010103
202510010104
202510010105
202510010106
202510010107
202510010108
202510010109
202510010110
202510010111
202510010112
202506010901
202506010902
202506010903
202506010904
202506010905
202506010906
202506010907
202506010908
202506010909
202506010910
202506010911
202506010912
202507010901
202507010902
202507010903
202507010904
202507010905
202507010906
202507010907
202507010908
202507010909
202507010910
202507010911
202507010912
202510010201
202510010202
202510010203
202510010204
202510010205
202510010206
202510010207
202510010208
202510010209
202510010210
202510010211
202510010212"""



# 改行で分割してリストに変換
numbers_list = race_id_list2.split("\n")

# 各数字にシングルクォーテーションを追加
race_id_list = [f'{num}' for num in numbers_list]



In [54]:
# raceページのhtmlをスクレイピング
html_paths_race = scraping.scrape_html_race(race_id_list=race_id_list, skip=True)

  6%|██▏                                | 1971/31346 [00:00<00:02, 10539.29it/s]

skipped: 201606010101
skipped: 201606010102
skipped: 201606010103
skipped: 201606010104
skipped: 201606010105
skipped: 201606010106
skipped: 201606010107
skipped: 201606010108
skipped: 201606010109
skipped: 201606010110
skipped: 201606010111
skipped: 201606010112
skipped: 201608010101
skipped: 201608010102
skipped: 201608010103
skipped: 201608010104
skipped: 201608010105
skipped: 201608010106
skipped: 201608010107
skipped: 201608010108
skipped: 201608010109
skipped: 201608010110
skipped: 201608010111
skipped: 201608010112
skipped: 201606010201
skipped: 201606010202
skipped: 201606010203
skipped: 201606010204
skipped: 201606010205
skipped: 201606010206
skipped: 201606010207
skipped: 201606010208
skipped: 201606010209
skipped: 201606010210
skipped: 201606010211
skipped: 201606010212
skipped: 201608010201
skipped: 201608010202
skipped: 201608010203
skipped: 201608010204
skipped: 201608010205
skipped: 201608010206
skipped: 201608010207
skipped: 201608010208
skipped: 201608010209
skipped: 2

 29%|█████████▉                         | 8936/31346 [00:00<00:00, 27141.27it/s]

skipped: 201610020207
skipped: 201610020208
skipped: 201610020209
skipped: 201610020210
skipped: 201610020211
skipped: 201610020212
skipped: 201601010201
skipped: 201601010202
skipped: 201601010203
skipped: 201601010204
skipped: 201601010205
skipped: 201601010206
skipped: 201601010207
skipped: 201601010208
skipped: 201601010209
skipped: 201601010210
skipped: 201601010211
skipped: 201601010212
skipped: 201604020301
skipped: 201604020302
skipped: 201604020303
skipped: 201604020304
skipped: 201604020305
skipped: 201604020306
skipped: 201604020307
skipped: 201604020308
skipped: 201604020309
skipped: 201604020310
skipped: 201604020311
skipped: 201604020312
skipped: 201610020301
skipped: 201610020302
skipped: 201610020303
skipped: 201610020304
skipped: 201610020305
skipped: 201610020306
skipped: 201610020307
skipped: 201610020308
skipped: 201610020309
skipped: 201610020310
skipped: 201610020311
skipped: 201610020312
skipped: 201601010301
skipped: 201601010302
skipped: 201601010303
skipped: 2

 55%|██████████████████▌               | 17148/31346 [00:00<00:00, 35336.00it/s]

...

 80%|███████████████████████████▏      | 25116/31346 [00:00<00:00, 37993.07it/s]

skipped: 202107010801
skipped: 202107010802
skipped: 202107010803
skipped: 202107010804
skipped: 202107010805
skipped: 202107010806
skipped: 202107010807
skipped: 202107010808
skipped: 202107010809
skipped: 202107010810
skipped: 202107010811
skipped: 202107010812
skipped: 202110010401
skipped: 202110010402
skipped: 202110010403
skipped: 202110010404
skipped: 202110010405
skipped: 202110010406
skipped: 202110010407
skipped: 202110010408
skipped: 202110010409
skipped: 202110010410
skipped: 202110010411
skipped: 202110010412
skipped: 202105010101
skipped: 202105010102
skipped: 202105010103
skipped: 202105010104
skipped: 202105010105
skipped: 202105010106
skipped: 202105010107
skipped: 202105010108
skipped: 202105010109
skipped: 202105010110
skipped: 202105010111
skipped: 202105010112
skipped: 202107010901
skipped: 202107010902
skipped: 202107010903
skipped: 202107010904
skipped: 202107010905
skipped: 202107010906
skipped: 202107010907
skipped: 202107010908
skipped: 202107010909
skipped: 2

 93%|███████████████████████████████▊  | 29289/31346 [00:00<00:00, 39159.88it/s]

skipped: 202304010609
skipped: 202304010610
skipped: 202304010611
skipped: 202304010612
skipped: 202305020901
skipped: 202305020902
skipped: 202305020903
skipped: 202305020904
skipped: 202305020905
skipped: 202305020906
skipped: 202305020907
skipped: 202305020908
skipped: 202305020909
skipped: 202305020910
skipped: 202305020911
skipped: 202305020912
skipped: 202308010901
skipped: 202308010902
skipped: 202308010903
skipped: 202308010904
skipped: 202308010905
skipped: 202308010906
skipped: 202308010907
skipped: 202308010908
skipped: 202308010909
skipped: 202308010910
skipped: 202308010911
skipped: 202308010912
skipped: 202304010701
skipped: 202304010702
skipped: 202304010703
skipped: 202304010704
skipped: 202304010705
skipped: 202304010706
skipped: 202304010707
skipped: 202304010708
skipped: 202304010709
skipped: 202304010710
skipped: 202304010711
skipped: 202304010712
skipped: 202305021001
skipped: 202305021002
skipped: 202305021003
skipped: 202305021004
skipped: 202305021005
skipped: 2

100%|█████████████████████████████████████| 31346/31346 [06:39<00:00, 78.40it/s]


In [ ]:
#エラーが起きたレースidについての処理

error_race_ids = ["201805050811"] 


import re
import time
import traceback
from pathlib import Path
from urllib.request import urlopen

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from urllib.request import urlopen, Request
from urllib.error import HTTPError




DATA_DIR = Path("..", "data")
HTML_RACE_DIR = DATA_DIR / "html" / "race"
HTML_HORSE_DIR = DATA_DIR / "html" / "horse"
HTML_PED_DIR = DATA_DIR / "html" / "ped"
HTML_LEADING_DIR = DATA_DIR / "html" / "leading"


def error_html_race(
    error_race_ids: list[str], save_dir: Path = HTML_RACE_DIR, skip: bool = True
) -> list[Path]:
    """
    netkeiba.comのraceページのhtmlをスクレイピングしてsave_dirに保存する関数。
    skip=Trueにすると、すでにhtmlが存在する場合はスキップされる。
    逆に上書きしたい場合は、skip=Falseにする。
    スキップされたhtmlのパスは返り値に含まれない。
    """
    updated_html_path_list = []
    save_dir.mkdir(parents=True, exist_ok=True)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

    for race_id in tqdm(error_race_ids):
        filepath = save_dir / f"{race_id}.bin"
        try:
            url = f"https://db.netkeiba.com/race/{race_id}"
            request = Request(url, headers=headers)
            html = urlopen(request).read()
            time.sleep(1)
            with open(filepath, "wb") as f:
                f.write(html)
            updated_html_path_list.append(filepath)
        except HTTPError as e:
            print(f"Error fetching {race_id}: {e}")

    return updated_html_path_list

error_paths_race = error_html_race(error_race_ids=error_race_ids)

In [ ]:
# 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
html_paths_race = [
    scraping.HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_id_list
]

In [129]:
html_paths_race = [
    scraping.HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_id_list
]

In [75]:
html_paths_race

[PosixPath('../data/html/race/202505010101.bin'),
 PosixPath('../data/html/race/202505010102.bin'),
 PosixPath('../data/html/race/202505010103.bin'),
 PosixPath('../data/html/race/202505010104.bin'),
 PosixPath('../data/html/race/202505010105.bin'),
 PosixPath('../data/html/race/202505010106.bin'),
 PosixPath('../data/html/race/202505010107.bin'),
 PosixPath('../data/html/race/202505010108.bin'),
 PosixPath('../data/html/race/202505010109.bin'),
 PosixPath('../data/html/race/202505010110.bin'),
 PosixPath('../data/html/race/202505010111.bin'),
 PosixPath('../data/html/race/202505010112.bin'),
 PosixPath('../data/html/race/202508010101.bin'),
 PosixPath('../data/html/race/202508010102.bin'),
 PosixPath('../data/html/race/202508010103.bin'),
 PosixPath('../data/html/race/202508010104.bin'),
 PosixPath('../data/html/race/202508010105.bin'),
 PosixPath('../data/html/race/202508010106.bin'),
 PosixPath('../data/html/race/202508010107.bin'),
 PosixPath('../data/html/race/202508010108.bin'),


In [130]:
results = create_rawdf.create_results(html_path_list=html_paths_race)

  0%|          | 0/31346 [00:00<?, ?it/s]

table not found at 201805010304
table not found at 202410020704
table not found at 202410020705
table not found at 202410020804
table not found at 202404010604
table not found at 202404010605
table not found at 202403020501
table not found at 202403020504
table not found at 202410030601
table not found at 202404030601
table not found at 202404030701
table not found at 202404030704
table not found at 202407020801
table not found at 202406050108
table not found at 202408070504
table not found at 202406050604
table not found at 202406050710
table not found at 202408070805
table not found at 202408070904
table not found at 202506010208
table not found at 202507010308
table not found at 202507010404
table not found at 202506010504
table not found at 202506010604
table not found at 202506010704
table not found at 202510010104
table not found at 202510010105
table not found at 202510010204


In [72]:
results

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,...,調教ﾀｲﾑ,厩舎ｺﾒﾝﾄ,備考,調教師,馬主,賞金(万円),horse_id,jockey_id,trainer_id,owner_id
0,202506010101,1,6,11,ピッツァナポリ,牝3,55.0,大野拓弥,1:12.2,NaN,...,NaN,NaN,NaN,[東] 新開幸一,吉田千津,560.0,2022105247,01096,01116,310007
1,202506010101,2,2,3,グリューネベルグ,牝3,55.0,丹内祐次,1:12.7,3,...,NaN,NaN,NaN,[東] 青木孝文,吉田和美,220.0,2022106581,01091,01156,138007
2,202506010101,3,7,13,ロッキンミー,牝3,52.0,佐藤翔馬,1:13.0,2,...,NaN,NaN,NaN,[東] 杉浦宏昭,石田暢子,140.0,2022107050,01207,01008,348034
3,202506010101,4,5,10,ルックアトミー,牝3,55.0,菊沢一樹,1:13.1,1/2,...,NaN,NaN,NaN,[東] 菊沢隆徳,是枝浩平,84.0,2022103727,01161,01115,370803
4,202506010101,5,6,12,ダイチラポール,牡3,54.0,長浜鴻緒,1:13.4,2,...,NaN,NaN,NaN,[東] 蛯名利弘,服部英利,56.0,2022100076,01214,01131,921031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,202510010212,7,5,5,ホウオウバローロ,牡4,57.0,斎藤新,1:47.1,2.1/2,...,NaN,NaN,NaN,[西] 音無秀孝,小笹芳央,NaN,2021105772,01178,01002,906033
7,202510010212,8,7,9,ピカレスクノベル,牡4,57.0,秋山稔樹,1:47.1,ハナ,...,NaN,NaN,NaN,[西] 高橋義忠,加藤徹,NaN,2021106087,01181,01129,036006
8,202510010212,9,8,11,アイファーファイト,牡6,58.0,藤懸貴志,1:47.3,3/4,...,NaN,NaN,NaN,[西] 鮫島一歩,中島稔,NaN,2019101233,01138,01046,119007
9,202510010212,10,7,8,ミスサラワク,牝4,52.0,柴田裕一,1:47.6,1.3/4,...,NaN,NaN,NaN,[東] 本間忍,乙訓史樹,NaN,2021105923,01212,01056,482031


In [59]:
#エラーが起きたidについての処理
import re


# race_id_list2 は改行で区切られた文字列
mess = """table not found at 201805010304
table not found at 201805050811
table not found at 202410020704
table not found at 202410020705
table not found at 202410020804
table not found at 202404010604
table not found at 202404010605


table not found at 202505010412
table not found at 202510010602
table not found at 202510010701
table not found at 202510010702
table not found at 202505010803
table not found at 202510011001"""

# 改行で分割してリストに変換
mess2 = mess.split("\n")

# 各数字にシングルクォーテーションを追加してリストに格納
messages = [f"{num}" for num in mess2]

# 正規表現を使って数字部分を抽出
missing_tables = [re.search(r'\d{12}', message).group() for message in messages]


In [ ]:
missing_tables

In [60]:
#エラーが起きたidについての処理2
import re
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np

DATA_DIR = Path("..", "data")
RAWDF_DIR = DATA_DIR / "rawdf"

# 例: 取得時にエラーが発生したrace_idのリスト
error_race_ids = missing_tables

dfs = {}

for error_race_id in tqdm(error_race_ids):
    try:
        # HTMLファイルのパスを指定
        html_path = DATA_DIR / "html" / "race" / f"{error_race_id}.bin"  
        with open(html_path, "rb") as f:
            html = (
                f.read()
                .replace(b"<diary_snap_cut>", b"")
                .replace(b"</diary_snap_cut>", b"")
            )
        
        soup = BeautifulSoup(html, "lxml").find("table", class_="race_table_01 nk_tb_common")
        
        # テーブルが見つからない場合はスキップ
        if soup is None:
            print(f"table not found for race_id: {error_race_id}")
            continue
        
        # HTML から DataFrame を抽出
        df = pd.read_html(html)[0]

        # horse_id, jockey_id, trainer_id, owner_id の抽出処理を関数化
        def extract_ids(soup, pattern, id_length):
            id_list = []
            a_list = soup.find_all("a", href=re.compile(pattern))
            for a in a_list:
                match = re.findall(rf"\d{{{id_length}}}", a["href"])
                id_list.append(match[0] if match else np.nan)
            
            # id_listの長さが df に満たない場合は NaN で埋める
            if len(id_list) < len(df):
                id_list.extend([np.nan] * (len(df) - len(id_list)))
            elif len(id_list) > len(df):
                id_list = id_list[:len(df)]
            return id_list
        
        df["horse_id"] = extract_ids(soup, r"^/horse/", 10)
        df["jockey_id"] = extract_ids(soup, r"^/jockey/", 5)
        df["trainer_id"] = extract_ids(soup, r"^/trainer/", 5)
        df["owner_id"] = extract_ids(soup, r"^/owner/", 6)

        # 最初の列にrace_idを挿入
        df.insert(0, "race_id", error_race_id)
        
        # レースIDごとに DataFrame を保存
        dfs[error_race_id] = df
    
    except IndexError as e:
        print(f"IndexError at {error_race_id}: {e}")
        continue
    except Exception as e:
        print(f"An error occurred for race_id {error_race_id}: {e}")
        continue


# すべての DataFrame を結合
concat_df = pd.concat(dfs.values())
concat_df.columns = concat_df.columns.str.replace(" ", "")
results = pd.concat([results, concat_df], ignore_index=True)

                    
# CSVを更新
def update_rawdf(
    new_df: pd.DataFrame,
    key: str,
    save_filename: str,
    save_dir: Path = RAWDF_DIR,
) -> None:
    """
    既存のrawdfに新しいデータを追加して保存する関数。
    """
    save_dir.mkdir(parents=True, exist_ok=True)
    if (save_dir / save_filename).exists():
        old_df = pd.read_csv(save_dir / save_filename, sep="\t", dtype={f"{key}": str})
        new_df[key] = new_df[key].astype(str)
        df = pd.concat([old_df[~old_df[key].isin(new_df[key])], new_df])
        df.to_csv(save_dir / save_filename, sep="\t", index=False)
    else:
        new_df.to_csv(save_dir / save_filename, sep="\t", index=False)


update_rawdf(
    concat_df,
    key="race_id",
    save_filename="results.csv",
    save_dir=RAWDF_DIR,
)

  0%|          | 0/87 [00:00<?, ?it/s]

table not found for race_id: 201805010304
IndexError at 202410020704: list index out of range
IndexError at 202410020705: list index out of range
IndexError at 202410020804: list index out of range
IndexError at 202404010604: list index out of range
IndexError at 202404010605: list index out of range
IndexError at 202403020501: list index out of range
IndexError at 202403020504: list index out of range
IndexError at 202410030601: list index out of range
IndexError at 202404030601: list index out of range
IndexError at 202404030701: list index out of range
IndexError at 202404030704: list index out of range
IndexError at 202407020801: list index out of range
IndexError at 202406050108: list index out of range
IndexError at 202408070504: list index out of range
IndexError at 202406050604: list index out of range
IndexError at 202406050710: list index out of range
IndexError at 202408070805: list index out of range
IndexError at 202408070904: list index out of range
IndexError at 20250601

In [175]:
results

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,ﾀｲﾑ指数,通過,上り,単勝,人気,馬体重,調教ﾀｲﾑ,厩舎ｺﾒﾝﾄ,備考,調教師,馬主,賞金(万円),horse_id,jockey_id,trainer_id,owner_id
0,202406010101,1,7,14,サンディブロンド,牝3,55.0,戸崎圭太,1:12.6,NaN,**,2-2,38.5,4.2,3.0,466(-12),NaN,NaN,NaN,[東] 田中博康,サンデーレーシング,550.0,2021105821,05386,01162,226800
1,202406010101,2,4,7,マイネルディレクト,牡3,57.0,丹内祐次,1:12.8,1,**,5-4,38.1,8.4,5.0,496(+6),NaN,NaN,NaN,[東] 高橋裕,サラブレッドクラブ・ラフィアン,220.0,2021106805,01091,00412,546800
2,202406010101,3,8,16,セイウンチドリ,牝3,55.0,三浦皇成,1:13.2,2.1/2,**,5-4,38.6,15.3,6.0,446(-6),NaN,NaN,NaN,[東] 村田一誠,西山茂行,140.0,2021102685,01122,01190,897009
3,202406010101,4,2,4,グリントリッター,セ3,57.0,石川裕紀,1:13.2,クビ,**,4-4,38.7,3.9,2.0,420(+2),NaN,NaN,NaN,[東] 深山雅史,ウエスト．フォレスト．ステイブル,83.0,2021100328,01150,01174,368803
4,202406010101,5,6,12,マニーブルース,牡3,57.0,吉田豊,1:13.3,1,**,8-7,38.1,50.9,9.0,458(+4),NaN,NaN,NaN,[東] 小手川準,サンライズ,55.0,2021102752,00733,01171,866800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46989,202408070602,12,2,4,マッハレイジング,牡2,56.0,岩田望来,1:29.7,1.1/4,**,3-5,41.1,64.9,8.0,474(-8),NaN,NaN,NaN,[西] 小崎憲,三浦勝仁,NaN,2022106597,01174,01085,810033
46990,202408070602,13,2,3,コパノジェームス,牡2,56.0,小崎綾也,1:30.1,2.1/2,**,12-11,40.4,557.2,15.0,504(+4),NaN,NaN,NaN,[西] 村山明,小林祥晃,NaN,2022100287,01152,01107,966007
46991,202408070602,14,3,5,レイリースピリット,牡2,54.0,吉村誠之,1:30.4,1.3/4,**,8-9,41.1,233.0,13.0,422(+4),NaN,NaN,NaN,[西] 浜田多実,山田裕二,NaN,2022103944,01216,01138,NaN
46992,202408070602,15,5,10,ナムラブルータス,牡2,55.0,田口貫太,1:31.9,9,**,16-16,40.1,130.6,10.0,482(+8),NaN,NaN,NaN,[西] 大橋勇樹,奈村睦弘,NaN,2022100144,01208,01065,972002


In [125]:
results = pd.read_csv(RAWDF_DIR / "results.csv", sep="\t")

In [73]:
# 重複チェック
results[["race_id", "horse_id"]].duplicated().sum()

np.int64(0)

In [61]:
race_info = create_rawdf.create_race_info(html_path_list=html_paths_race)

  0%|          | 0/31346 [00:00<?, ?it/s]

'NoneType' object has no attribute 'find' at 201805010304


In [62]:
#レース情報テーブルの作成
"""
*注意・create_rawdf.create_race_infoは２度コードを実行すると、return_table.csvに列がどんどん増えていくため、一度の実行にすること
エラーが起きている場合、一度return_table.csvを削除してから行うこと
欠損などで２度実行したい場合は、一度、csvファイルを読み込み、列をまとめてから実行すること
最終状態はまとめた状態にすること
"""
#一回一回、csvファイルを消さないと空欄の列が追加される
return_tables = create_rawdf.create_return_tables(html_path_list=html_paths_race)

  0%|          | 0/31346 [00:00<?, ?it/s]

table not found at 201805010303
table not found at 202410020703
table not found at 202410020703
table not found at 202410020803
table not found at 202404010603
table not found at 202404010603
table not found at 202402011012
table not found at 202403020503
table not found at 202403020612
table not found at 202401020512
table not found at 202401020612
table not found at 202404030703
table not found at 202404030812
table not found at 202406050107
table not found at 202408070503
table not found at 202406050603
table not found at 202406050709
table not found at 202408070804
table not found at 202408070903
table not found at 202506010207
table not found at 202507010307
table not found at 202507010403
table not found at 202506010503
table not found at 202506010603
table not found at 202506010703
table not found at 202510010103
table not found at 202510010103
table not found at 202510010203


In [63]:
#エラー処理
log_text = """
table not found at 201805010303
table not found at 202410020703
table not found at 202410020703
table not found at 202410020803
table not found at 202404010603
table not found at 202404010603
table not found at 202402011012
table not found at 202403020503
table not found at 202403020612
table not found at 202401020512
table not found at 202401020612
table not found at 202404030703
table not found at 202404030812
table not found at 202406050107
table not found at 202408070503
table not found at 202406050603
table not found at 202406050709
table not found at 202408070804
table not found at 202408070903
table not found at 202506010207
table not found at 202507010307
table not found at 202507010403
table not found at 202506010503
table not found at 202506010603
table not found at 202506010703
table not found at 202510010103
table not found at 202510010103
table not found at 202510010203
"""
import re
race_ids = sorted(set(re.findall(r'table not found at (\d+)', log_text)))
html_paths_race_error = [HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_ids]



In [64]:
#エラー処理続き
#一回一回、csvファイルを消さないと空欄の列が追加される
return_tables3 = create_rawdf.create_return_tables(html_path_list=html_paths_race_error)

  0%|          | 0/25 [00:00<?, ?it/s]

In [67]:
#列が二重になった場合の処理
#欠損処理をした場合、列が二重になるためここで修正
DATA_DIR = Path("..", "data")
RAWDF_DIR = DATA_DIR / "rawdf"
df = pd.read_csv(RAWDF_DIR/ "return_tables.csv", sep="\t")
# 欠損値を埋める処理
for col in ['0', '1', '2', '3']:  # 列名を文字列として扱う
    col1 = f"{col}.1"  # '0.1', '1.1', '2.1', '3.1'
    df[col] = df[col].combine_first(df[col1])

# 不要な列（0.1, 1.1, 2.1, 3.1）を削除
df.drop(columns=['0.1', '1.1', '2.1', '3.1'], inplace=True)

# 必要なら保存
df.to_csv(RAWDF_DIR/"return_tables.csv", index=False, sep="\t")
#列が二重になった場合の処理
#欠損処理をした場合、列が二重になるためここで修正
# DATA_DIR = Path("..", "data")
# RAWDF_DIR = DATA_DIR / "rawdf"
# df = pd.read_csv(RAWDF_DIR/ "return_tables.csv", sep="\t")
# # 欠損値を埋める処理
# for col in ['0', '1', '2', '3']:  # 列名を文字列として扱う
#     col1 = f"{col}.2"  # '0.1', '1.1', '2.1', '3.1'
#     df[col] = df[col].combine_first(df[col1])

# # 不要な列（0.1, 1.1, 2.1, 3.1）を削除
# df.drop(columns=['0.2', '1.2', '2.2', '3.2'], inplace=True)

# # 必要なら保存
# df.to_csv(RAWDF_DIR/"return_tables.csv", index=False, sep="\t")

/var/folders/43/k8vtkf1x6js1yc8cj0ml9g2h0000gn/T/ipykernel_2052/857406092.py:20: DtypeWarning: Columns (5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAWDF_DIR/ "return_tables.csv", sep="\t")


In [76]:
results_new = create_rawdf.create_results(
    html_path_list=html_paths_race,
    save_filename = "results_new.csv",
)
#results_newは毎回消す

  0%|          | 0/288 [00:00<?, ?it/s]

table not found at 202505010412
table not found at 202510010602
table not found at 202510010701
table not found at 202510010702
table not found at 202505010803
table not found at 202510011001


In [77]:
#エラーが起きたidについての処理
import re


# race_id_list2 は改行で区切られた文字列
mess = """table not found at 202505010412
table not found at 202510010602
table not found at 202510010701
table not found at 202510010702
table not found at 202505010803
table not found at 202510011001"""

# 改行で分割してリストに変換
mess2 = mess.split("\n")

# 各数字にシングルクォーテーションを追加してリストに格納
messages = [f"{num}" for num in mess2]

# 正規表現を使って数字部分を抽出
missing_tables = [re.search(r'\d{12}', message).group() for message in messages]


In [79]:
#エラーが起きたidについての処理2
import re
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np

DATA_DIR = Path("..", "data")
RAWDF_DIR = DATA_DIR / "rawdf"

# 例: 取得時にエラーが発生したrace_idのリスト
error_race_ids = missing_tables

dfs = {}

for error_race_id in tqdm(error_race_ids):
    try:
        # HTMLファイルのパスを指定
        html_path = DATA_DIR / "html" / "race" / f"{error_race_id}.bin"  
        with open(html_path, "rb") as f:
            html = (
                f.read()
                .replace(b"<diary_snap_cut>", b"")
                .replace(b"</diary_snap_cut>", b"")
            )
        
        soup = BeautifulSoup(html, "lxml").find("table", class_="race_table_01 nk_tb_common")
        
        # テーブルが見つからない場合はスキップ
        if soup is None:
            print(f"table not found for race_id: {error_race_id}")
            continue
        
        # HTML から DataFrame を抽出
        df = pd.read_html(html)[0]

        # horse_id, jockey_id, trainer_id, owner_id の抽出処理を関数化
        def extract_ids(soup, pattern, id_length):
            id_list = []
            a_list = soup.find_all("a", href=re.compile(pattern))
            for a in a_list:
                match = re.findall(rf"\d{{{id_length}}}", a["href"])
                id_list.append(match[0] if match else np.nan)
            
            # id_listの長さが df に満たない場合は NaN で埋める
            if len(id_list) < len(df):
                id_list.extend([np.nan] * (len(df) - len(id_list)))
            elif len(id_list) > len(df):
                id_list = id_list[:len(df)]
            return id_list
        
        df["horse_id"] = extract_ids(soup, r"^/horse/", 10)
        df["jockey_id"] = extract_ids(soup, r"^/jockey/", 5)
        df["trainer_id"] = extract_ids(soup, r"^/trainer/", 5)
        df["owner_id"] = extract_ids(soup, r"^/owner/", 6)

        # 最初の列にrace_idを挿入
        df.insert(0, "race_id", error_race_id)
        
        # レースIDごとに DataFrame を保存
        dfs[error_race_id] = df
    
    except IndexError as e:
        print(f"IndexError at {error_race_id}: {e}")
        continue
    except Exception as e:
        print(f"An error occurred for race_id {error_race_id}: {e}")
        continue


# すべての DataFrame を結合
concat_df = pd.concat(dfs.values())
concat_df.columns = concat_df.columns.str.replace(" ", "")
results_new = pd.concat([results_new, concat_df], ignore_index=True)

                    
# CSVを更新
def update_rawdf(
    new_df: pd.DataFrame,
    key: str,
    save_filename: str,
    save_dir: Path = RAWDF_DIR,
) -> None:
    """
    既存のrawdfに新しいデータを追加して保存する関数。
    """
    save_dir.mkdir(parents=True, exist_ok=True)
    if (save_dir / save_filename).exists():
        old_df = pd.read_csv(save_dir / save_filename, sep="\t", dtype={f"{key}": str})
        new_df[key] = new_df[key].astype(str)
        df = pd.concat([old_df[~old_df[key].isin(new_df[key])], new_df])
        df.to_csv(save_dir / save_filename, sep="\t", index=False)
    else:
        new_df.to_csv(save_dir / save_filename, sep="\t", index=False)


update_rawdf(
    concat_df,
    key="race_id",
    save_filename="results_new.csv",
    save_dir=RAWDF_DIR,
)

  0%|          | 0/6 [00:00<?, ?it/s]

In [80]:
#馬の過去成績をスクレイピング
horse_id_list = results_new["horse_id"].unique()
len(horse_id_list)

3516

In [ ]:
horse_id_list 

In [81]:
html_paths_horse = scraping.scrape_html_horse(
    horse_id_list=horse_id_list, skip=False
)

100%|█████████████████████████████████████| 3516/3516 [1:18:54<00:00,  1.35s/it]


Errors occurred for the following horse_ids:
[]


In [141]:
# 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
html_paths_horse = [
    scraping.HTML_HORSE_DIR / f"{horse_id}.bin" for horse_id in horse_id_list
]

In [142]:
html_paths_horse = list(scraping.HTML_HORSE_DIR.glob("*.bin"))

In [87]:
# 馬の過去成績テーブルの作成
horse_results = create_rawdf.create_horse_results(html_paths_horse)

  0%|          | 0/3516 [00:00<?, ?it/s]

In [179]:
#エラーが起きたbinファイルをもう一度取得し、再度作成する
log_text_horse = """
table not found at 2022110143
table not found at 2022110145
table not found at 2022110151
table not found at 2022110152
table not found at 2022110153
"""

horse_ids = sorted(set(re.findall(r'table not found at (\d+)', log_text_horse)))

html_paths_horse_error = scraping.scrape_html_horse(
    horse_id_list=horse_ids, skip=False
)
#上書きしたいのでFalse

['2022110143', '2022110145', '2022110151', '2022110152', '2022110153']

In [ ]:
html_paths_horse_error = [HTML_HORSE_DIR / f"{horse_id}.bin" for horse_id in horse_ids]

In [ ]:
horse_results3 = create_rawdf.create_horse_results(html_paths_horse_error)

In [102]:
#エラー処理、rank_1racesノーデータ
error_horse = [2013104922, 2012102111, 2013102667, 2013103000, 2013103225, 2013105080, 2013104156, 2013104851, 2013104067, 2013104168, 2013101047, 2013103589, 2013104825, 2013103865, 2013102726, 2013104155, 2013102892, 2010190006, 2013100877, 2012105460, 2014104434, 2014104429, 2013104823, 2013102706, 2014101055, 2014101782, 2014102055, 2014104335, 2014100427, 2013100446, 2014100465, 2014101483, 2014103307, 2014101559, 2014109117, 2014106369, 2013103940, 2014103604, 2014101893, 2014105200, 2014104277, 2014102709, 2013106459, 2012103911, 2013110132, 2014101961, 2014101850, 2012109107, 2014101699, 2014104300, 2014104078, 2012190003, 2014102046, 2014101019, 2013102669, 2014105315, 2014106466, 2014103934, 2014104911, 2014101725, 2013110131, 2014104142, 2014106387, 2014103991, 2014106396, 2014101434, 2014104932, 2014104582, 2014103016, 2011100019, 2014101254, 2013102309, 2014100801, 2011190004, 2014104011, 2015102248, 2014103890, 2014102925, 2014103856, 2013101112, 2015101465, 2015104624, 2015100129, 2015103679, 2015101899, 2014101785, 2014100706, 2015100887, 2015101468, 2015103255, 2015100664, 2015101952, 2015100993, 2014100662, 2015109104, 2015109102, 2014104202, 2011100189, 2014102468, 2014105213, 2014101408, 2015106125, 2015102341, 2015102113, 2015103800, 2015101086, 2014101674, 2015101106, 2011190005, 2015100072, 2015103959, 2015101459, 2014101507, 2013190002, 2012190004, 2011190006, 2015100080, 2015100316, 2015101103, 2015102805, 2015103214, 2015100313, 2015100660, 2014105252, 2015105501, 2015102184, 2014101371, 2015105578, 2015105585, 2015101953, 2015101105, 2015105684, 2015100910, 2015101044, 2015102783, 2014101708, 2015103972, 2015100150, 2015109059, 2015106271, 2015105849, 2015102098, 2013103350, 2014104830, 2012190005, 2011190007, 2015105398, 2015106502, 2015101831, 2016105439, 2016105669, 2015101946, 2016103387, 2016106387, 2016106082, 2016101433, 2016105508, 2016102640, 2016103958, 2016105718, 2016103825, 2016102960, 2016106473, 2016100394, 2015100966, 2015104115, 2015103588, 2014103705, 2016102275, 2016104043, 2016102966, 2016105191, 2016101826, 2016101757, 2016103900, 2016103779, 2016101117, 2013190003, 2014190001, 2016102564, 2016103049, 2013103629, 2016101195, 2016102847, 2016103017, 2015100106, 2016103578, 2016101390, 2016100893, 2016105664, 2016101708, 2016101135, 2016103179, 2016104103, 2016101971, 2016102736, 2016103423, 2016104192, 2016101815, 2015100061, 2015103884, 2016103741, 2017103509, 2017102160, 2017100161, 2015102718, 2014103479, 2014104623, 2017100315, 2017100098, 2017106483, 2017103017, 2017101519, 2017102126, 2016101009, 2017102195, 2017100320, 2017105883, 2017103403, 2017101109, 2017104539, 2017106043, 2017109131, 2017102016, 2017102186, 2017101506, 2017101509, 2017100492, 2017102697, 2017103340, 2017106690, 2016100981, 2015106351, 2016102967, 2017100297, 2017103152, 2015105971, 2016104785, 2017106497, 2014100235, 2017100323, 2017103195, 2017100495, 2017102562, 2015102904, 2017101611, 2014102775, 2017104112, 2017110165, 2017106486, 2017100498, 2017106481, 2017104467, 2017105745, 2016100930, 2017102820, 2015110058, 2016100945, 2017110058, 2017101695, 2018103851, 2016100503, 2018102961, 2018101319, 2018106228, 2018105796, 2018104021, 2017103090, 2018105961, 2018102733, 2018101935, 2018104233, 2018103678, 2018106270, 2018102399, 2018109122, 2018109169, 2018103618, 2018100520, 2018100548, 2018100753, 2018105726, 2018102735, 2018103231, 2017101060, 2018104545, 2018106358, 2018105978, 2017105803, 2018100342, 2017103367, 2018102309, 2018104523, 2018100353, 2018100141, 2018104117, 2013190004, 2018105595, 2018104028, 2018105644, 2016102502, 2017104319, 2017100908, 2018102366, 2018106653, 2018102819, 2018105750, 2018100331, 2018104127, 2018105956, 2017106044, 2017102070, 2018105636, 2017106475, 2017101741, 2017106224, 2016102002, 2015103718, 2017104438, 2019105982, 2018109009, 2019100295, 2019110102, 2019102106, 2019101340, 2019101137, 2019103624, 2019100270, 2019100455, 2019109134, 2019109116, 2018102344, 2019105587, 2019102183, 2019101859, 2018100479, 2018104674, 2017106419, 2019109090, 2019106957, 2016190001, 2016190002, 2016190003, 2018103507, 2018105654, 2018100526, 2019100886, 2017100391, 2018103898, 2019103469, 2019103451, 2019103439, 2017100216, 2019100123, 2019106053, 2018105014, 2018101468, 2019106147, 2019106757, 2018100116, 2018101321, 2019104202, 2019100721, 2020100873, 2020105729, 2020102411, 2020104280, 2019105765, 2020107146, 2020106446, 2018106578, 2020100686, 2020109061, 2020103834, 2020100101, 2020109140, 2020109149, 2020109133, 2020109156, 2020109139, 2019104175, 2020102493, 2019100398, 2020100248, 2020102412, 2020104873, 2020101532, 2020105574, 2020101320, 2018103515, 2018106559, 2019104307, 2020104882, 2019190001, 2019190002, 2019190004, 2019190003, 2020100654, 2020105042, 2020105701, 2020106959, 2020102352, 2020102319, 2020106453, 2019105183, 2020100435, 2020100304, 2020105288, 2020100121, 2020102123, 2020104731, 2020109003, 2019101529, 2020100443, 2019100268, 2019106183, 2020101755, 2019106611, 2020102274, 2020102431, 2021101069, 2021102532, 2021104666, 2021104368, 2021104027, 2021109009, 2021102388, 2021102868, 2021102296, 2021100444, 2021109161, 2021109174, 2021109130, 2021109158, 2021103695, 2021109022, 2021106264, 2021106510, 2021103761, 2021101955, 2020102387, 2020105737, 2021102332, 2021101916, 2020103338, 2021107138, 2021106372, 2021105203, 2021100102, 2017190002, 2021102155, 2021105204, 2020106249, 2020103404, 2021100121, 2021100384, 2021106730, 2019103013, 2021100447, 2021103136, 2020104285, 2018105352, 2021104167, 2021101386, 2021105237, 2021101385, 2020106751, 2020103867, 2019102665, 2021102904, 2021102224, 2018190001, 2021105056, 2021107173, 2021105864, 2021107180, 2021102224, 2021103557, 2021105864, 2021100143, 2020101242, 2021102224, 2020101115, 2018190002, 2018190003, 2021100143, 2020110050, 2021100055, 2021101134, 2021107180, 2021105102, 2020102238, 2021102904, 2021107173, 2021103557, 2022105718, 2021101619, 2020102775, 2022109171, 2022109107, 2022109155, 2021107173, 2021103259, 2021105056, 2022110015, 2022106874, 2022100789, 2021100867, 2022100161, 2022102614, 2022104509, 2022101909, 2021103557, 2021101488, 2022103220, 2022109024, 2022106498, 2022103355, 2022106104, 2022100855, 2021101134, 2022105163, 2022104771, 2021100143, 2022104772, 2022104720, 2022105830, 2022100161, 2022102794, 2022102086, 2022107209, 2022107304, 2021107180, 2021105864, 2022102849, 2022106394, 2022104028, 2021105102, 2021100143, 2021110090, 2021105056, 2022110108, 2022106874, 2022103995, 2022105590, 2021107180, 2021103557, 2018190004, 2022104751, 2022104898, 2022102617, 2021110090, 2022105590, 2022105718, 2022110014, 2022101745, 2021103557, 2021104931, 2022100286, 2021101134, 2021107173, 2021105574, 2022104994, 2022101197, 2022105402, 2022104812, 2021105680, 2022101732, 2022106394, 2022104929, 2022102849, 2021101487, 2022105421, 2022105291, 2022105891, 2022102148, 2021107173, 2020106694, 2022104771, 2022105102, 2022104720, 2022105951, 2021105102, 2021101487, 2020190002, 2021105864, 2018190005, 2022103995, 2022102217, 2022100874, 2021105574, 2020190005, 2020190003, 2020190004, 2022104832, 2022103288, 2021102904, 2020103864, 2021100143, 2021105214, 2022110012, 2022100982]


In [103]:
# 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
horse_results_none_pre = [
    scraping.HTML_HORSE_DIR / f"{n}.bin" for n in error_horse
]

In [104]:
horse_results_none = create_rawdf.create_horse_results(horse_results_none_pre)

  0%|          | 0/602 [00:00<?, ?it/s]

2013104922 は新馬のレビューがあるためスキップ
2012102111 は新馬のレビューがあるためスキップ
2013102667 は新馬のレビューがあるためスキップ
2013103000 は新馬のレビューがあるためスキップ
2013103225 は新馬のレビューがあるためスキップ
2013105080 は新馬のレビューがあるためスキップ
2013104156 は新馬のレビューがあるためスキップ
2013104851 は新馬のレビューがあるためスキップ
2013104067 は新馬のレビューがあるためスキップ
2013104168 は新馬のレビューがあるためスキップ
2013101047 は新馬のレビューがあるためスキップ
2013103589 は新馬のレビューがあるためスキップ
2013104825 は新馬のレビューがあるためスキップ
2013103865 は新馬のレビューがあるためスキップ
2013102726 は新馬のレビューがあるためスキップ
2013104155 は新馬のレビューがあるためスキップ
2013102892 は新馬のレビューがあるためスキップ
2010190006 は新馬のレビューがあるためスキップ
2013100877 は新馬のレビューがあるためスキップ
2012105460 は新馬のレビューがあるためスキップ
2014104434 は新馬のレビューがあるためスキップ
2014104429 は新馬のレビューがあるためスキップ
2013104823 は新馬のレビューがあるためスキップ
2013102706 は新馬のレビューがあるためスキップ
2014101055 は新馬のレビューがあるためスキップ
2014101782 は新馬のレビューがあるためスキップ
2014102055 は新馬のレビューがあるためスキップ
2014104335 は新馬のレビューがあるためスキップ
2014100427 は新馬のレビューがあるためスキップ
2013100446 は新馬のレビューがあるためスキップ
2014100465 は新馬のレビューがあるためスキップ
2014101483 は新馬のレビューがあるためスキップ
2014103307 は新馬のレビューがあるためスキップ
2014101559 は新馬のレビューがあるためスキップ
2014109117 は新馬

In [83]:
#血統情報の取得
if "results" not in locals():
    results = pd.read_csv(create_rawdf.RAWDF_DIR / "results.csv", sep="\t")
horse_id_list = results["horse_id"].unique()

In [84]:
html_paths_ped = scraping.scrape_html_ped(
    horse_id_list=horse_id_list,
    save_dir=Path(HTML_PED_DIR)
)


  1%|▎                                    | 483/50368 [00:00<00:20, 2473.30it/s]

skipped: 2013100513
skipped: 2013102845
skipped: 2013102932
skipped: 2013103416
skipped: 2013100107
skipped: 2013103101
skipped: 2013102277
skipped: 2013102283
skipped: 2013106151
skipped: 2013104555
skipped: 2013104382
skipped: 2013100103
skipped: 2013103702
skipped: 2013104573
skipped: 2013104668
skipped: 2013105621
skipped: 2013100001
skipped: 2013101293
skipped: 2013101939
skipped: 2013101957
skipped: 2013105676
skipped: 2013100653
skipped: 2013105713
skipped: 2013102919
skipped: 2013105125
skipped: 2013101182
skipped: 2013104107
skipped: 2013103597
skipped: 2013106344
skipped: 2013109101
skipped: 2013100549
skipped: 2013101914
skipped: 2011102141
skipped: 2011101687
skipped: 2012100798
skipped: 2011102712
skipped: 2011105378
skipped: 2012101907
skipped: 2011105789
skipped: 2011101394
skipped: 2011104171
skipped: 2011104597
skipped: 2012105911
skipped: 2012102008
skipped: 2012102781
skipped: 2011104863
skipped: 2012105673
skipped: 2011104104
skipped: 2011103670
skipped: 2012102961


  3%|█▏                                  | 1623/50368 [00:00<00:10, 4854.12it/s]

skipped: 2008102799
skipped: 2007100476
skipped: 2009103759
skipped: 2009105899
skipped: 2011104166
skipped: 2010102243
skipped: 2012104737
skipped: 2009105122
skipped: 2009104585
skipped: 2012100724
skipped: 2009102606
skipped: 2011104417
skipped: 2008100608
skipped: 2009106572
skipped: 2010103608
skipped: 2009102195
skipped: 2010101972
skipped: 2009103539
skipped: 2011102826
skipped: 2010104886
skipped: 2009102658
skipped: 2010101569
skipped: 2010104562
skipped: 2012105979
skipped: 2006100429
skipped: 2012101787
skipped: 2012103711
skipped: 2012104115
skipped: 2011101385
skipped: 2012101712
skipped: 2011103876
skipped: 2012106134
skipped: 2012103950
skipped: 2011102452
skipped: 2012103142
skipped: 2011105191
skipped: 2012105688
skipped: 2011102246
skipped: 2012106080
skipped: 2012100717
skipped: 2011103994
skipped: 2012109197
skipped: 2013100998
skipped: 2013102154
skipped: 2013101748
skipped: 2013110069
skipped: 2013106403
skipped: 2013109026
skipped: 2013103143
skipped: 2013105320


 11%|███▊                               | 5507/50368 [00:00<00:03, 13678.32it/s]

...

skipped: 2018106545
skipped: 2018101808
skipped: 2018104578
skipped: 2018101586
skipped: 2018102375
skipped: 2018105267
skipped: 2018106565
skipped: 2018101299
skipped: 2018105470
skipped: 2018102552
skipped: 2018105929
skipped: 2018103810
skipped: 2018106661
skipped: 2018103452
skipped: 2018106454
skipped: 2018104818
skipped: 2018100985
skipped: 2018106446
skipped: 2018103019
skipped: 2018101540
skipped: 2018102543
skipped: 2018101236
skipped: 2018100817
skipped: 2018101183
skipped: 2018106586
skipped: 2018103568
skipped: 2018101124
skipped: 2018106199
skipped: 2018109074
skipped: 2018106203
skipped: 2018105199
skipped: 2018103736
skipped: 2018104765
skipped: 2018105387
skipped: 2018101791
skipped: 2018105435
skipped: 2018104550
skipped: 2018100070
skipped: 2018102575
skipped: 2018105057
skipped: 2018103433
skipped: 2018106067
skipped: 2018104210
skipped: 2018104726
skipped: 2018100168
skipped: 2018102492
skipped: 2018110090
skipped: 2018104670
skipped: 2018100822
skipped: 2018101335


 90%|██████████████████████████████▌   | 45366/50368 [00:01<00:00, 25198.60it/s]

skipped: 2020106656
skipped: 2021100527
skipped: 2021100821
skipped: 2021100692
skipped: 2021107234
skipped: 2021104705
skipped: 2020101560
skipped: 2020103377
skipped: 2021104724
skipped: 2021106540
skipped: 2021105885
skipped: 2021102685
skipped: 2021105555
skipped: 2021105688
skipped: 2021106102
skipped: 2021104995
skipped: 2021103976
skipped: 2021105524
skipped: 2021100065
skipped: 2021106488
skipped: 2021103342
skipped: 2021105309
skipped: 2021106687
skipped: 2021104475
skipped: 2020103569
skipped: 2020101808
skipped: 2020105937
skipped: 2021105559
skipped: 2021105283
skipped: 2021105867
skipped: 2021105598
skipped: 2021110063
skipped: 2021105374
skipped: 2021106683
skipped: 2020103478
skipped: 2021100356
skipped: 2021103277
skipped: 2021106347
skipped: 2021104734
skipped: 2021104910
skipped: 2021102029
skipped: 2020103032
skipped: 2020101849
skipped: 2021104094
skipped: 2021107169
skipped: 2021103325
skipped: 2021104127
skipped: 2021103052
skipped: 2021105297
skipped: 2021104659


100%|██████████████████████████████████| 50368/50368 [00:01<00:00, 25319.42it/s]

skipped: 2021101827
skipped: 2021107264
skipped: 2021104034
skipped: 2021103234
skipped: 2021100945
skipped: 2021102214
skipped: 2021101609
skipped: 2021101134
skipped: 2021105505
skipped: 2021104500
skipped: 2021104097
skipped: 2021107035
skipped: 2021105881
skipped: 2021101672
skipped: 2021101019
skipped: 2021106077
skipped: 2021105260
skipped: 2021106704
skipped: 2021104677
skipped: 2021101207
skipped: 2021103919
skipped: 2021104566
skipped: 2021107270
skipped: 2021105427
skipped: 2021110077
skipped: 2021104988
skipped: 2021100905
skipped: 2021103032
skipped: 2021102549
skipped: 2021110098
skipped: 2021101349
skipped: 2021104971
skipped: 2021102715
skipped: 2021103002
skipped: 2021105847
skipped: 2021100256
skipped: 2021107384
skipped: 2021101264
skipped: 2021105498
skipped: 2021106380
skipped: 2021104159
skipped: 2021103208
skipped: 2021101893
skipped: 2021103163
skipped: 2021107000
skipped: 2021100179
skipped: 2021102200
skipped: 2021102803
skipped: 2021107191
skipped: 2021103455


In [ ]:
#エラーが起きた場合の処理
import csv
from pathlib import Path

# CSVファイルのパスを指定
error_log_path = Path('../data/html/ped/error_horse_ids.csv')

# CSVファイルからhorse_idをリストに読み込み
with open(error_log_path, "r", newline="") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # ヘッダーをスキップ
    error_horse_ids_list = [row[0] for row in reader]

# 読み込んだエラーのhorse_idを表
error_horse_ids_list = list(map(int, error_horse_ids_list))
error_horse_ids_list = np.array(error_horse_ids_list)
error_html_paths_ped = scraping.scrape_html_ped(
    horse_id_list=error_horse_ids_list,
    save_dir=Path(HTML_PED_DIR)
)

In [ ]:
# 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
html_paths_ped = [
    scraping.HTML_PED_DIR / f"{horse_id}.bin" for horse_id in horse_id_list
]

In [86]:
html_paths_ped

([], [])

In [85]:
# 血統テーブルの作成
peds = create_rawdf.create_peds(html_paths_ped)

  0%|          | 0/2 [00:00<?, ?it/s]

TypeError: expected str, bytes or os.PathLike object, not list

In [ ]:
html_paths_ped

In [34]:
#リーディング情報の取得/騎手
for year in [2015,2016,2017,2018,2019,2020,2021,2022, 2023,2024]:
    _ = scraping.scrape_html_leading(
        leading_type="jockey_leading",
        year=year,
        pages=[1, 2, 3, 4],
    )
    
# 2017年のみ5ページ目まである
_ = scraping.scrape_html_leading(
    leading_type="jockey_leading",
    year=2017,
    pages=[1, 2, 3, 4, 5],
)

100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 3225.77it/s]


skipped: ../data/html/leading/jockey_leading/2015_01.bin
skipped: ../data/html/leading/jockey_leading/2015_02.bin
skipped: ../data/html/leading/jockey_leading/2015_03.bin
skipped: ../data/html/leading/jockey_leading/2015_04.bin


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 4523.38it/s]


skipped: ../data/html/leading/jockey_leading/2016_01.bin
skipped: ../data/html/leading/jockey_leading/2016_02.bin
skipped: ../data/html/leading/jockey_leading/2016_03.bin
skipped: ../data/html/leading/jockey_leading/2016_04.bin


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 3721.65it/s]


...

skipped: ../data/html/leading/jockey_leading/2023_01.bin
skipped: ../data/html/leading/jockey_leading/2023_02.bin
skipped: ../data/html/leading/jockey_leading/2023_03.bin
skipped: ../data/html/leading/jockey_leading/2023_04.bin


100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 14550.92it/s]


skipped: ../data/html/leading/jockey_leading/2024_01.bin
skipped: ../data/html/leading/jockey_leading/2024_02.bin
skipped: ../data/html/leading/jockey_leading/2024_03.bin
skipped: ../data/html/leading/jockey_leading/2024_04.bin


100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 7505.91it/s]

skipped: ../data/html/leading/jockey_leading/2017_01.bin
skipped: ../data/html/leading/jockey_leading/2017_02.bin
skipped: ../data/html/leading/jockey_leading/2017_03.bin
skipped: ../data/html/leading/jockey_leading/2017_04.bin
skipped: ../data/html/leading/jockey_leading/2017_05.bin


In [35]:
# スクレイピングしたhtmlファイルのパスを取得
html_paths_jockey_leading = list(
    (scraping.HTML_LEADING_DIR / "jockey_leading").glob("*.bin")
)
# rawデータの作成
jockey_leading = create_rawdf.create_jockey_leading(html_paths_jockey_leading)

  0%|          | 0/41 [00:00<?, ?it/s]

In [36]:
#調教師
for year in [2015,2016,2017,2018,2019,2020,2021,2022, 2023,2024]:
    _ = scraping.scrape_html_leading(
        leading_type="trainer_leading",
        year=year,
        pages=[1, 2, 3, 4, 5],
    )
    
# 2018年のみ6ページ目まである
_ = scraping.scrape_html_leading(
    leading_type="trainer_leading",
    year=2018,
    pages=[1, 2, 3, 4, 5, 6],
)

100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 6362.72it/s]


skipped: ../data/html/leading/trainer_leading/2015_01.bin
skipped: ../data/html/leading/trainer_leading/2015_02.bin
skipped: ../data/html/leading/trainer_leading/2015_03.bin
skipped: ../data/html/leading/trainer_leading/2015_04.bin
skipped: ../data/html/leading/trainer_leading/2015_05.bin


100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 9450.89it/s]


skipped: ../data/html/leading/trainer_leading/2016_01.bin
skipped: ../data/html/leading/trainer_leading/2016_02.bin
skipped: ../data/html/leading/trainer_leading/2016_03.bin
skipped: ../data/html/leading/trainer_leading/2016_04.bin
skipped: ../data/html/leading/trainer_leading/2016_05.bin


100%|██████████████████████████████████████████| 5/5 [00:00<00:00, 13477.84it/s]


...

skipped: ../data/html/leading/trainer_leading/2023_01.bin
skipped: ../data/html/leading/trainer_leading/2023_02.bin
skipped: ../data/html/leading/trainer_leading/2023_03.bin
skipped: ../data/html/leading/trainer_leading/2023_04.bin
skipped: ../data/html/leading/trainer_leading/2023_05.bin


100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 6630.26it/s]


skipped: ../data/html/leading/trainer_leading/2024_01.bin
skipped: ../data/html/leading/trainer_leading/2024_02.bin
skipped: ../data/html/leading/trainer_leading/2024_03.bin
skipped: ../data/html/leading/trainer_leading/2024_04.bin
skipped: ../data/html/leading/trainer_leading/2024_05.bin


100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 10490.13it/s]

skipped: ../data/html/leading/trainer_leading/2018_01.bin
skipped: ../data/html/leading/trainer_leading/2018_02.bin
skipped: ../data/html/leading/trainer_leading/2018_03.bin
skipped: ../data/html/leading/trainer_leading/2018_04.bin
skipped: ../data/html/leading/trainer_leading/2018_05.bin
skipped: ../data/html/leading/trainer_leading/2018_06.bin


In [37]:
# スクレイピングしたhtmlファイルのパスを取得
html_paths_trainer_leading = list(
    (scraping.HTML_LEADING_DIR / "trainer_leading").glob("*.bin")
)
# rawデータの作成
trainer_leading = create_rawdf.create_trainer_leading(html_paths_trainer_leading)

  0%|          | 0/51 [00:00<?, ?it/s]

In [38]:
#種牡馬
for year in range(2015, 2025):
    _ = scraping.scrape_html_leading(
        leading_type="sire_leading",
        year=year,
        pages=range(1, 10),
    )

100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 14358.59it/s]


skipped: ../data/html/leading/sire_leading/2015_01.bin
skipped: ../data/html/leading/sire_leading/2015_02.bin
skipped: ../data/html/leading/sire_leading/2015_03.bin
skipped: ../data/html/leading/sire_leading/2015_04.bin
skipped: ../data/html/leading/sire_leading/2015_05.bin
skipped: ../data/html/leading/sire_leading/2015_06.bin
skipped: ../data/html/leading/sire_leading/2015_07.bin
skipped: ../data/html/leading/sire_leading/2015_08.bin
skipped: ../data/html/leading/sire_leading/2015_09.bin


100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 4573.39it/s]


skipped: ../data/html/leading/sire_leading/2016_01.bin
skipped: ../data/html/leading/sire_leading/2016_02.bin
skipped: ../data/html/leading/sire_leading/2016_03.bin
skipped: ../data/html/leading/sire_leading/2016_04.bin
skipped: ../data/html/leading/sire_leading/2016_05.bin
skipped: ../data/html/leading/sire_leading/2016_06.bin
skipped: ../data/html/leading/sire_leading/2016_07.bin
skipped: ../data/html/leading/sire_leading/2016_08.bin
skipped: ../data/html/leading/sire_leading/2016_09.bin


100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 4275.06it/s]


...

skipped: ../data/html/leading/sire_leading/2022_01.bin
skipped: ../data/html/leading/sire_leading/2022_02.bin
skipped: ../data/html/leading/sire_leading/2022_03.bin
skipped: ../data/html/leading/sire_leading/2022_04.bin
skipped: ../data/html/leading/sire_leading/2022_05.bin
skipped: ../data/html/leading/sire_leading/2022_06.bin
skipped: ../data/html/leading/sire_leading/2022_07.bin
skipped: ../data/html/leading/sire_leading/2022_08.bin
skipped: ../data/html/leading/sire_leading/2022_09.bin


100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 21076.90it/s]


skipped: ../data/html/leading/sire_leading/2023_01.bin
skipped: ../data/html/leading/sire_leading/2023_02.bin
skipped: ../data/html/leading/sire_leading/2023_03.bin
skipped: ../data/html/leading/sire_leading/2023_04.bin
skipped: ../data/html/leading/sire_leading/2023_05.bin
skipped: ../data/html/leading/sire_leading/2023_06.bin
skipped: ../data/html/leading/sire_leading/2023_07.bin
skipped: ../data/html/leading/sire_leading/2023_08.bin
skipped: ../data/html/leading/sire_leading/2023_09.bin


100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 18706.01it/s]

skipped: ../data/html/leading/sire_leading/2024_01.bin
skipped: ../data/html/leading/sire_leading/2024_02.bin
skipped: ../data/html/leading/sire_leading/2024_03.bin
skipped: ../data/html/leading/sire_leading/2024_04.bin
skipped: ../data/html/leading/sire_leading/2024_05.bin
skipped: ../data/html/leading/sire_leading/2024_06.bin
skipped: ../data/html/leading/sire_leading/2024_07.bin
skipped: ../data/html/leading/sire_leading/2024_08.bin
skipped: ../data/html/leading/sire_leading/2024_09.bin


In [39]:
# スクレイピングしたhtmlファイルのパスを取得
html_paths_sire_leading = list(
    (scraping.HTML_LEADING_DIR / "sire_leading").glob("*.bin")
)

# rawデータの作成
sire_leading = create_rawdf.create_sire_leading(html_paths_sire_leading)

  0%|          | 0/90 [00:00<?, ?it/s]

In [40]:
for year in range(2015, 2025):
    _ = scraping.scrape_html_leading(
        leading_type="bms_leading",
        year=year,
        pages=range(1, 10),
    )

100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 9933.88it/s]


skipped: ../data/html/leading/bms_leading/2015_01.bin
skipped: ../data/html/leading/bms_leading/2015_02.bin
skipped: ../data/html/leading/bms_leading/2015_03.bin
skipped: ../data/html/leading/bms_leading/2015_04.bin
skipped: ../data/html/leading/bms_leading/2015_05.bin
skipped: ../data/html/leading/bms_leading/2015_06.bin
skipped: ../data/html/leading/bms_leading/2015_07.bin
skipped: ../data/html/leading/bms_leading/2015_08.bin
skipped: ../data/html/leading/bms_leading/2015_09.bin


100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 13960.33it/s]


skipped: ../data/html/leading/bms_leading/2016_01.bin
skipped: ../data/html/leading/bms_leading/2016_02.bin
skipped: ../data/html/leading/bms_leading/2016_03.bin
skipped: ../data/html/leading/bms_leading/2016_04.bin
skipped: ../data/html/leading/bms_leading/2016_05.bin
skipped: ../data/html/leading/bms_leading/2016_06.bin
skipped: ../data/html/leading/bms_leading/2016_07.bin
skipped: ../data/html/leading/bms_leading/2016_08.bin
skipped: ../data/html/leading/bms_leading/2016_09.bin


100%|██████████████████████████████████████████| 9/9 [00:00<00:00, 10503.27it/s]


...

skipped: ../data/html/leading/bms_leading/2022_01.bin
skipped: ../data/html/leading/bms_leading/2022_02.bin
skipped: ../data/html/leading/bms_leading/2022_03.bin
skipped: ../data/html/leading/bms_leading/2022_04.bin
skipped: ../data/html/leading/bms_leading/2022_05.bin
skipped: ../data/html/leading/bms_leading/2022_06.bin
skipped: ../data/html/leading/bms_leading/2022_07.bin
skipped: ../data/html/leading/bms_leading/2022_08.bin
skipped: ../data/html/leading/bms_leading/2022_09.bin


100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 7037.42it/s]


skipped: ../data/html/leading/bms_leading/2023_01.bin
skipped: ../data/html/leading/bms_leading/2023_02.bin
skipped: ../data/html/leading/bms_leading/2023_03.bin
skipped: ../data/html/leading/bms_leading/2023_04.bin
skipped: ../data/html/leading/bms_leading/2023_05.bin
skipped: ../data/html/leading/bms_leading/2023_06.bin
skipped: ../data/html/leading/bms_leading/2023_07.bin
skipped: ../data/html/leading/bms_leading/2023_08.bin
skipped: ../data/html/leading/bms_leading/2023_09.bin


100%|███████████████████████████████████████████| 9/9 [00:00<00:00, 8197.34it/s]

skipped: ../data/html/leading/bms_leading/2024_01.bin
skipped: ../data/html/leading/bms_leading/2024_02.bin
skipped: ../data/html/leading/bms_leading/2024_03.bin
skipped: ../data/html/leading/bms_leading/2024_04.bin
skipped: ../data/html/leading/bms_leading/2024_05.bin
skipped: ../data/html/leading/bms_leading/2024_06.bin
skipped: ../data/html/leading/bms_leading/2024_07.bin
skipped: ../data/html/leading/bms_leading/2024_08.bin
skipped: ../data/html/leading/bms_leading/2024_09.bin


In [41]:
# スクレイピングしたhtmlファイルのパスを取得
html_paths_bms_leading = list(
    (scraping.HTML_LEADING_DIR / "bms_leading").glob("*.bin")
)

# rawデータの作成
bms_leading = create_rawdf.create_bms_leading(html_paths_bms_leading)

  0%|          | 0/90 [00:00<?, ?it/s]

### 実際に予測する際の事前準備

- 実際の予測時の処理
- 予測したいレース前日などに出走馬が発表されたら、以下の事前準備を行う

In [3]:
# 予測母集団の作成
#予測したい日を入力してください
prediction_population = create_prediction_population.create(kaisai_date="20250323")

scraping race_id_list...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:21<00:00, 81.93s/it]

scraping horse_id_list...


  0%|          | 0/36 [00:00<?, ?it/s]

In [4]:
prediction_population 

,date,race_id,horse_id
0,2025-03-23,202506020801,2022100392
1,2025-03-23,202506020801,2022101641
2,2025-03-23,202506020801,2022104383
3,2025-03-23,202506020801,2022100306
4,2025-03-23,202506020801,2022107233
...,...,...,...
13,2025-03-23,202507020412,2019105487
14,2025-03-23,202507020412,2021102578
15,2025-03-23,202507020412,2021102235
16,2025-03-23,202507020412,2021105068


In [5]:
# 当日出走馬のhorse_idリスト
horse_id_list = prediction_population["horse_id"].unique()

In [6]:
## 馬の過去成績取得 更新している可能性が高いため
html_paths_horse = scraping.scrape_html_horse(horse_id_list=horse_id_list, skip=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 533/533 [11:17<00:00,  1.27s/it]


Errors occurred for the following horse_ids:
[]


In [7]:
html_paths_horse 

[PosixPath('../data/html/horse/2022100392.bin'),
 PosixPath('../data/html/horse/2022101641.bin'),
 PosixPath('../data/html/horse/2022104383.bin'),
 PosixPath('../data/html/horse/2022100306.bin'),
 PosixPath('../data/html/horse/2022107233.bin'),
 PosixPath('../data/html/horse/2022102229.bin'),
 PosixPath('../data/html/horse/2022102991.bin'),
 PosixPath('../data/html/horse/2022101481.bin'),
 PosixPath('../data/html/horse/2022107074.bin'),
 PosixPath('../data/html/horse/2022100682.bin'),
 PosixPath('../data/html/horse/2022102108.bin'),
 PosixPath('../data/html/horse/2022100949.bin'),
 PosixPath('../data/html/horse/2022104350.bin'),
 PosixPath('../data/html/horse/2022103774.bin'),
 PosixPath('../data/html/horse/2022103994.bin'),
 PosixPath('../data/html/horse/2022100763.bin'),
 PosixPath('../data/html/horse/2022102036.bin'),
 PosixPath('../data/html/horse/2022102021.bin'),
 PosixPath('../data/html/horse/2022106512.bin'),
 PosixPath('../data/html/horse/2022101022.bin'),
 PosixPath('../data/

In [ ]:
#エラー修正

In [70]:
# バイナリファイルの読み込み例
with open(HTML_HORSE_DIR /'2022101314.bin', 'rb') as file:
    data = file.read()  

In [71]:
df = pd.read_html(data)[2]
df
#取れてない方

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2024/11/30,4中京1,晴,8,3歳以上1勝クラス,NaN,16,3,6,11.2,...,2.8,**,4-8-9,34.6-35.8,37.5,586(+14),NaN,NaN,ダズリングブレイヴ,NaN
1,2024/06/29,3小倉1,曇,8,3歳以上1勝クラス,NaN,12,1,1,17.1,...,1.0,**,1-1-1-2,35.1-37.0,38.0,572(+6),NaN,NaN,カズミクラーシュ,NaN
2,2024/03/16,1阪神7,晴,11,若葉S(L),NaN,9,8,9,36.3,...,3.5,**,3-3-2-2,36.1-34.6,37.9,566(0),NaN,NaN,ミスタージーティー,NaN
3,2024/01/28,2京都2,晴,5,3歳新馬,NaN,13,4,4,4.8,...,0.0,**,2-2-2-2,37.6-35.6,35.6,566(0),NaN,NaN,(ジャスティンダイス),620.0


In [4]:
# バイナリファイルの読み込み例
with open(HTML_HORSE_DIR /'2022100896.bin', 'rb') as file:
    data2 = file.read()  

In [21]:
df2 = pd.read_html(data2)[2]
df2

#取れてる方

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2024/10/19,4東京5,晴,9,アイビーS(L),NaN,8,1,1,41.2,...,1.9,**,6-6-6,35.7-34.3,34.5,404(-6),NaN,NaN,マスカレードボール,NaN
1,2024/09/22,4中山7,曇,5,2歳新馬,NaN,14,8,13,23.5,...,-0.3,**,1-1-1-1,37.3-34.8,34.8,410(0),NaN,NaN,(アルティテュード),720.0


In [87]:
df.columns[0]

np.int64(0)

In [80]:
from pathlib import Path  # PosixPath を使うために pathlib をインポート

# 当日出走馬の過去成績テーブル作成
horse_results_error = create_rawdf.create_horse_results(
    html_path_list=[Path(HTML_HORSE_DIR) /'2021102904.bin'],  # リストとして渡す
    save_filename="horse_results_error.csv",
)


  0%|          | 0/1 [00:00<?, ?it/s]

In [81]:
horse_results_error

,horse_id,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
0,2021102904,2024/11/30,4中京1,晴,8,3歳以上1勝クラス,NaN,16,3,6,11.2,5,14,鮫島克駿,57,芝1600,良,**,1:36.3,2.8,**,4-8-9,34.6-35.8,37.5,586(+14),NaN,NaN,ダズリングブレイヴ,NaN
1,2021102904,2024/06/29,3小倉1,曇,8,3歳以上1勝クラス,NaN,12,1,1,17.1,4,7,西村淳也,55,芝1800,良,**,1:48.9,1.0,**,1-1-1-2,35.1-37.0,38.0,572(+6),NaN,NaN,カズミクラーシュ,NaN
2,2021102904,2024/03/16,1阪神7,晴,11,若葉S(L),NaN,9,8,9,36.3,8,9,池添謙一,57,芝2000,良,**,2:03.2,3.5,**,3-3-2-2,36.1-34.6,37.9,566(0),NaN,NaN,ミスタージーティー,NaN
3,2021102904,2024/01/28,2京都2,晴,5,3歳新馬,NaN,13,4,4,4.8,2,1,池添謙一,57,芝2000,良,**,2:03.6,0.0,**,2-2-2-2,37.6-35.6,35.6,566(0),NaN,NaN,(ジャスティンダイス),620.0


In [8]:
# 当日出走馬の過去成績テーブル作成
horse_results_prediction = create_rawdf.create_horse_results(
    html_path_list=html_paths_horse,
    save_filename="horse_results_prediction.csv",
)

  0%|          | 0/533 [00:00<?, ?it/s]

2022104383 の HTML に3つ目のテーブルがありません
2022103774 の HTML に3つ目のテーブルがありません
2022100763 の HTML に3つ目のテーブルがありません
2022102036 の HTML に3つ目のテーブルがありません
2022104147 の HTML に3つ目のテーブルがありません
2022101402 の HTML に3つ目のテーブルがありません
2022106726 の HTML に3つ目のテーブルがありません
2022105373 の HTML に3つ目のテーブルがありません
2022104519 の HTML に3つ目のテーブルがありません
2022105817 の HTML に3つ目のテーブルがありません
2022102293 の HTML に3つ目のテーブルがありません
2022100368 の HTML に3つ目のテーブルがありません
2022104679 の HTML に3つ目のテーブルがありません
2022105215 の HTML に3つ目のテーブルがありません
2022100967 の HTML に3つ目のテーブルがありません
2022103463 の HTML に3つ目のテーブルがありません
2022106697 の HTML に3つ目のテーブルがありません
2022103268 の HTML に3つ目のテーブルがありません
2022107397 の HTML に3つ目のテーブルがありません
2022103558 の HTML に3つ目のテーブルがありません
2022106561 の HTML に3つ目のテーブルがありません
2022105536 の HTML に3つ目のテーブルがありません
2022102277 の HTML に3つ目のテーブルがありません
2022101570 の HTML に3つ目のテーブルがありません


In [9]:
#血統データ取得
# 当日出走馬の血統をスクレイピング
html_paths_peds = scraping.scrape_html_ped(
    horse_id_list=horse_id_list,
    save_dir=Path(HTML_PED_DIR)
)

  1%|▌                                                                                                            | 3/533 [00:00<00:30, 17.34it/s]

skipped: 2022100392
skipped: 2022101641
skipped: 2022100306
skipped: 2022107233
skipped: 2022102229
skipped: 2022102991
skipped: 2022101481
skipped: 2022107074
skipped: 2022100682
skipped: 2022102108
skipped: 2022100949
skipped: 2022104350


  3%|██▊                                                                                                         | 14/533 [00:00<00:25, 20.55it/s]

skipped: 2022103994


  3%|███▋                                                                                                        | 18/533 [00:01<01:05,  7.82it/s]

...

skipped: 2022106203


 79%|████████████████████████████████████████████████████████████████████████████████████▌                      | 421/533 [00:10<00:04, 26.92it/s]

skipped: 2022106194
skipped: 2022101426
skipped: 2022103584
skipped: 2022106216
skipped: 2022106775
skipped: 2022101605
skipped: 2022101397
skipped: 2022106830
skipped: 2022106807
skipped: 2022107085
skipped: 2022105091
skipped: 2022105272
skipped: 2022106245
skipped: 2022103025
skipped: 2022103679
skipped: 2022105214
skipped: 2022106938
skipped: 2022110029
skipped: 2022102312
skipped: 2022103905
skipped: 2020107109
skipped: 2020104052
skipped: 2019101624
skipped: 2021102982
skipped: 2021105894
skipped: 2021103484
skipped: 2019105457
skipped: 2021100090
skipped: 2020102876
skipped: 2019105420
skipped: 2021103720
skipped: 2021100680
skipped: 2021105939
skipped: 2021103943
skipped: 2021105875
skipped: 2021102754
skipped: 2020103625
skipped: 2019100689
skipped: 2020100652
skipped: 2021104039
skipped: 2020103901
skipped: 2021100730


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 533/533 [00:11<00:00, 47.36it/s]

skipped: 2021105897
skipped: 2021106370
skipped: 2019106972
skipped: 2020105495
skipped: 2021104622
skipped: 2021105680
skipped: 2020102956
skipped: 2020103933
skipped: 2021102406
skipped: 2020103055
skipped: 2020101635
skipped: 2021101659
skipped: 2021100573
skipped: 2021101279
skipped: 2019101753
skipped: 2020102952
skipped: 2021104658
skipped: 2021101056
skipped: 2021101240
skipped: 2021106091
skipped: 2020104000
skipped: 2021100739
skipped: 2021102797
skipped: 2021105432
skipped: 2020104336
skipped: 2022105474
skipped: 2022104678
skipped: 2022103262
skipped: 2022102963
skipped: 2022106756
skipped: 2022104648
skipped: 2022104694
skipped: 2022107342
skipped: 2022104748
skipped: 2022100293
skipped: 2021100329
skipped: 2021105984
skipped: 2020100987
skipped: 2021104824
skipped: 2021102388
skipped: 2020104790
skipped: 2021105929
skipped: 2020105670
skipped: 2021104448
skipped: 2020103045
skipped: 2021100845
skipped: 2019102265
skipped: 2020103414
skipped: 2021101259
skipped: 2021106164


In [10]:
# skipされたものも含めて全てのhtmlファイルのパスを取得
html_paths_peds = [scraping.HTML_PED_DIR / f"{horse_id}.bin" for horse_id in horse_id_list]

In [11]:
# 当日出走馬の血統テーブル作成
peds_prediction = create_rawdf.create_peds(
    html_path_list=html_paths_peds,
    save_filename="peds_prediction.csv",
)

  0%|          | 0/533 [00:00<?, ?it/s]

### 馬場状態のための過去データ収集

In [16]:
TMP_DIR2 = scraping.DATA_DIR / "tmp_predict"

In [17]:
# 開催日一覧の取得、行うレースの月を含む1ヶ月前をとっておく、実際に使用するのは12日前のレースのみ
kaisai_date_list_prediction = scraping.scrape_kaisai_date(
    from_="2025-03", to_="2025-03", save_dir=TMP_DIR2
)

100%|████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.21s/it]


In [18]:
kaisai_date_list_prediction

['20250301',
 '20250302',
 '20250308',
 '20250309',
 '20250315',
 '20250316',
 '20250322',
 '20250323',
 '20250330']

In [19]:
cut_index = kaisai_date_list_prediction.index("20250322")

# リストをスライスして削除
kaisai_date_list_prediction = kaisai_date_list_prediction[:cut_index]

In [20]:
kaisai_date_list_prediction

['20250301', '20250302', '20250308', '20250309', '20250315', '20250316']

In [23]:
TMP_DIR3 = scraping.DATA_DIR / "tmp_predict2"
#一回一回消す

In [24]:
# スクレイピング対象レースのid取得
race_id_list_prediction = scraping.scrape_race_id_list(
    kaisai_date_list_prediction, save_dir=TMP_DIR3
)

  0%|                                                                                        | 0/6 [00:00<?, ?it/s]

skipped: 202506020101
skipped: 202506020102
skipped: 202506020103
skipped: 202506020104
skipped: 202506020105
skipped: 202506020106
skipped: 202506020107
skipped: 202506020108
skipped: 202506020109
skipped: 202506020110
skipped: 202506020111
skipped: 202506020112
skipped: 202509010101
skipped: 202509010102
skipped: 202509010103
skipped: 202509010104
skipped: 202509010105
skipped: 202509010106
skipped: 202509010107
skipped: 202509010108
skipped: 202509010109
skipped: 202509010110
skipped: 202509010111
skipped: 202509010112
skipped: 202510011101
skipped: 202510011102
skipped: 202510011103
skipped: 202510011104
skipped: 202510011105
skipped: 202510011106
skipped: 202510011107
skipped: 202510011108
skipped: 202510011109
skipped: 202510011110


 17%|█████████████▎                                                                  | 1/6 [01:22<06:51, 82.31s/it]

skipped: 202510011111
skipped: 202510011112
skipped: 202506020201
skipped: 202506020202
skipped: 202506020203
skipped: 202506020204
skipped: 202506020205
skipped: 202506020206
skipped: 202506020207
skipped: 202506020208
skipped: 202506020209
skipped: 202506020210
skipped: 202506020211
skipped: 202506020212
skipped: 202509010201
skipped: 202509010202
skipped: 202509010203
skipped: 202509010204
skipped: 202509010205
skipped: 202509010206
skipped: 202509010207
skipped: 202509010208
skipped: 202509010209
skipped: 202509010210
skipped: 202509010211
skipped: 202509010212


 33%|██████████████████████████▋                                                     | 2/6 [01:25<02:21, 35.49s/it]

...

 50%|████████████████████████████████████████                                        | 3/6 [01:27<01:00, 20.26s/it]

skipped: 202509010307
skipped: 202509010308
skipped: 202509010309
skipped: 202509010310
skipped: 202509010311
skipped: 202509010312
skipped: 202506020401
skipped: 202506020402
skipped: 202506020403
skipped: 202506020404
skipped: 202506020405
skipped: 202506020406
skipped: 202506020407
skipped: 202506020408
skipped: 202506020409
skipped: 202506020410
skipped: 202506020411
skipped: 202506020412
skipped: 202509010401
skipped: 202509010402
skipped: 202509010403
skipped: 202509010404
skipped: 202509010405
skipped: 202509010406
skipped: 202509010407
skipped: 202509010408
skipped: 202509010409
skipped: 202509010410
skipped: 202509010411


 67%|█████████████████████████████████████████████████████▎                          | 4/6 [01:29<00:26, 13.35s/it]

skipped: 202509010412


100%|████████████████████████████████████████████████████████████████████████████████| 6/6 [01:34<00:00, 15.68s/it]


In [25]:
race_id_list_prediction

['202506020101',
 '202506020102',
 '202506020103',
 '202506020104',
 '202506020105',
 '202506020106',
 '202506020107',
 '202506020108',
 '202506020109',
 '202506020110',
 '202506020111',
 '202506020112',
 '202509010101',
 '202509010102',
 '202509010103',
 '202509010104',
 '202509010105',
 '202509010106',
 '202509010107',
 '202509010108',
 '202509010109',
 '202509010110',
 '202509010111',
 '202509010112',
 '202510011101',
 '202510011102',
 '202510011103',
 '202510011104',
 '202510011105',
 '202510011106',
 '202510011107',
 '202510011108',
 '202510011109',
 '202510011110',
 '202510011111',
 '202510011112',
 '202506020201',
 '202506020202',
 '202506020203',
 '202506020204',
 '202506020205',
 '202506020206',
 '202506020207',
 '202506020208',
 '202506020209',
 '202506020210',
 '202506020211',
 '202506020212',
 '202509010201',
 '202509010202',
 '202509010203',
 '202509010204',
 '202509010205',
 '202509010206',
 '202509010207',
 '202509010208',
 '202509010209',
 '202509010210',
 '202509010211

In [26]:
# raceページのhtmlをスクレイピング
html_paths_race_prediction = scraping_prediction.scrape_html_race(race_id_list=race_id_list_prediction, skip = False)

  0%|          | 0/192 [00:00<?, ?it/s]

In [27]:
html_paths_race_prediction

[PosixPath('../data/html/race2/202506020101.bin'),
 PosixPath('../data/html/race2/202506020102.bin'),
 PosixPath('../data/html/race2/202506020103.bin'),
 PosixPath('../data/html/race2/202506020104.bin'),
 PosixPath('../data/html/race2/202506020105.bin'),
 PosixPath('../data/html/race2/202506020106.bin'),
 PosixPath('../data/html/race2/202506020107.bin'),
 PosixPath('../data/html/race2/202506020108.bin'),
 PosixPath('../data/html/race2/202506020109.bin'),
 PosixPath('../data/html/race2/202506020110.bin'),
 PosixPath('../data/html/race2/202506020111.bin'),
 PosixPath('../data/html/race2/202506020112.bin'),
 PosixPath('../data/html/race2/202509010101.bin'),
 PosixPath('../data/html/race2/202509010102.bin'),
 PosixPath('../data/html/race2/202509010103.bin'),
 PosixPath('../data/html/race2/202509010104.bin'),
 PosixPath('../data/html/race2/202509010105.bin'),
 PosixPath('../data/html/race2/202509010106.bin'),
 PosixPath('../data/html/race2/202509010107.bin'),
 PosixPath('../data/html/race2/

In [ ]:
# # 途中で処理が途切れるなどした場合は、直接htmlのファイルパスを取得
# html_paths_race_prediction = [
#     scraping_prediction.HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_id_list_prediction
# ]

In [28]:
results_pre = scraping_prediction.create_results(html_path_list=html_paths_race_prediction)

  0%|          | 0/192 [00:00<?, ?it/s]

table not found at 202510011104
table not found at 202510011105
table not found at 202510011202
table not found at 202510011203
table not found at 202510011204
table not found at 202509010401
table not found at 202506020502
table not found at 202506020603


In [30]:
import re

# メッセージリスト
messages = [
"table not found at 202509010401",
"table not found at 202506020502",
"table not found at 202506020603",


]

# 正規表現で数字部分を抽出
missing_tables = [re.search(r'\d{12}', message).group() for message in messages]

# 抽出したリストを表示
print(missing_tables)


['202509010401', '202506020502', '202506020603']


In [31]:
#エラー引き起こしたレースIDを追加
import re
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

DATA_DIR = Path("..", "data")
RAWDF_DIR = DATA_DIR / "rawdf2"

# 例: 取得時にエラーが発生したrace_idのリスト
# error_race_ids =missing_tables
error_race_ids =missing_tables
# 実際のエラーのrace_idをリストに格納

dfs = {}

# エラーのrace_idリストをループ処理
for error_race_id in tqdm(error_race_ids):
    try:
        html_path = DATA_DIR / "html" / "race2" / f"{error_race_id}.bin"  # 対応するHTMLファイルのパス
        with open(html_path, "rb") as f:
            
            race_id = html_path.stem
            html = (
                f.read()
                .replace(b"<diary_snap_cut>", b"")
                .replace(b"</diary_snap_cut>", b"")
            )
            soup = BeautifulSoup(html, "lxml").find(
                "table", class_="race_table_01 nk_tb_common"
            )
            df = pd.read_html(html)[0]

        # horse_id列追加
        horse_id_list = []
        a_list = soup.find_all("a", href=re.compile(r"^/horse/"))
        for a in a_list:
            horse_id = re.findall(r"\d{10}", a["href"])[0]
            horse_id_list.append(horse_id)
        df["horse_id"] = horse_id_list

        # jockey_id列追加
        jockey_id_list = []
        a_list = soup.find_all("a", href=re.compile(r"^/jockey/"))
        for a in a_list:
            jockey_id = re.findall(r"\d{5}", a["href"])[0]
            jockey_id_list.append(jockey_id)
        df["jockey_id"] = jockey_id_list

        # trainer_id列追加
        trainer_id_list = []
        a_list = soup.find_all("a", href=re.compile(r"^/trainer/"))
        for a in a_list:
            trainer_id = re.findall(r"\d{5}", a["href"])[0]
            trainer_id_list.append(trainer_id)
        df["trainer_id"] = trainer_id_list

        # 最初の列にrace_idを挿入
        df.insert(0, "race_id", error_race_id)
        dfs[error_race_id] = df
        # owner_id列追加
        owner_id_list = []
        a_list = soup.find_all("a", href=re.compile(r"^/owner/"))
        for a in a_list:
            owner_id = re.findall(r"\d{6}", a["href"])[0]
            owner_id_list.append(owner_id)
        df["owner_id"] = owner_id_list

    except IndexError:
        print(f"table not found at {error_race_id}")
        continue


# 全てのデータフレームを結合
concat_df = pd.concat(dfs.values())
concat_df.columns = concat_df.columns.str.replace(" ", "")

# 結果を保存
update_rawdf(
    concat_df,
    key="race_id",
    save_filename="results.csv",
    save_dir=RAWDF_DIR,
)
def update_rawdf(
    new_df: pd.DataFrame,
    key: str,
    save_filename: str,
    save_dir: Path = RAWDF_DIR,
) -> None:
    """
    既存のrawdfに新しいデータを追加して保存する関数。
    """
    save_dir.mkdir(parents=True, exist_ok=True)
    if (save_dir / save_filename).exists():
        old_df = pd.read_csv(save_dir / save_filename, sep="\t", dtype={f"{key}": str})
        # 念の為、key列をstr型に変換
        new_df[key] = new_df[key].astype(str)
        df = pd.concat([old_df[~old_df[key].isin(new_df[key])], new_df])
        df.to_csv(save_dir / save_filename, sep="\t", index=False)
    else:
        # ファイルが存在しない場合は単にそのまま保存
        new_df.to_csv(save_dir / save_filename, sep="\t", index=False)
# error_race_id = error_race_ids[0]
# html_path = DATA_DIR /"html"/ "race2"/ f"{error_race_id}.bin"  # 対応するHTMLファイルのパス
# with open(html_path, "rb") as f:
#     html = (
#         f.read()
#         .replace(b"<diary_snap_cut>", b"")
#         .replace(b"</diary_snap_cut>", b"")
#     )
#     soup = BeautifulSoup(html, "lxml").find(
#         "table", class_="race_table_01 nk_tb_common"
#     )
#     df = pd.read_html(html)[0]


# # エラーのrace_idリストをループ処理
# # for error_race_id in tqdm(error_race_ids):
# #     try:
# #         html_path = DATA_DIR /"html"/ "race"/ f"{error_race_ids}.bin"   # 対応するHTMLファイルのパス
# #         with open(html_path, "rb") as f:
# #             html = (
# #                 f.read()
# #                 .replace(b"<diary_snap_cut>", b"")
# #                 .replace(b"</diary_snap_cut>", b"")
# #             )
# #             soup = BeautifulSoup(html, "lxml").find(
# #                 "table", class_="race_table_01 nk_tb_common"
# #             )
# #             df = pd.read_html(html)[0]

#             # horse_id列追加
# horse_id_list = []
# a_list = soup.find_all("a", href=re.compile(r"^/horse/"))
# for a in a_list:
#     horse_id = re.findall(r"\d{10}", a["href"])[0]
#     horse_id_list.append(horse_id)
# df["horse_id"] = horse_id_list

# # jockey_id列追加
# jockey_id_list = []
# a_list = soup.find_all("a", href=re.compile(r"^/jockey/"))
# for a in a_list:
#     jockey_id = re.findall(r"\d{5}", a["href"])[0]
#     jockey_id_list.append(jockey_id)
# df["jockey_id"] = jockey_id_list

# # trainer_id列追加
# trainer_id_list = []
# a_list = soup.find_all("a", href=re.compile(r"^/trainer/"))
# for a in a_list:
#     trainer_id = re.findall(r"\d{5}", a["href"])[0]
#     trainer_id_list.append(trainer_id)
# df["trainer_id"] = trainer_id_list

# # # owner_id列追加
# # owner_id_list = []
# # a_list = soup.find_all("a", href=re.compile(r"^/owner/"))
# # for a in a_list:
# #     owner_id = re.findall(r"\d{6}", a["href"])[0]
# #     owner_id_list.append(owner_id)
# # df["owner_id"] = owner_id_list
# import numpy as np

# # a_list = soup.find_all("a", href=re.compile(r"^/owner/"))
# # for a in a_list:
# #     match = re.findall(r"\d{6}", a["href"])
# #     if match:
# #         owner_id = match[0]
# #     else:
# #         owner_id = np.nan  # 空の場合は NaN を挿入
# #     owner_id_list.append(owner_id)

# # # owner_id_list の長さを df の行数に合わせる
# # if len(owner_id_list) < len(df):
# #     # 足りない分を NaN で埋める
# #     owner_id_list.extend([np.nan] * (len(df) - len(owner_id_list)))
# # elif len(owner_id_list) > len(df):
# #     # 多い場合は df の行数に合わせて切り詰める
# #     owner_id_list = owner_id_list[:len(df)]

# # df["owner_id"] = owner_id_list


# # 最初の列にrace_idを挿入
# df.insert(0, "race_id", error_race_id)
# dfs[error_race_id] = df


# # except IndexError as e:
# #     print(f"table not found at {error_race_id}")
# #     continue
# # except Exception as e:
# #     print(f"An error occurred for race_id {error_race_id}: {e}")
# #     continue

# def update_rawdf(
#     new_df: pd.DataFrame,
#     key: str,
#     save_filename: str,
#     save_dir: Path = RAWDF_DIR,
# ) -> None:
#     """
#     既存のrawdfに新しいデータを追加して保存する関数。
#     """
#     save_dir.mkdir(parents=True, exist_ok=True)
#     if (save_dir / save_filename).exists():
#         old_df = pd.read_csv(save_dir / save_filename, sep="\t", dtype={f"{key}": str})
#         # 念の為、key列をstr型に変換
#         new_df[key] = new_df[key].astype(str)
#         df = pd.concat([old_df[~old_df[key].isin(new_df[key])], new_df])
#         df.to_csv(save_dir / save_filename, sep="\t", index=False)
#     else:
#         # ファイルが存在しない場合は単にそのまま保存
#         new_df.to_csv(save_dir / save_filename, sep="\t", index=False)


# concat_df = pd.concat(dfs.values())
# concat_df.columns = concat_df.columns.str.replace(" ", "")
# update_rawdf(
#     concat_df,
#     key="race_id",
#     save_filename="results.csv",
#     save_dir=RAWDF_DIR,
# )
df

  0%|          | 0/3 [00:00<?, ?it/s]

table not found at 202509010401
table not found at 202506020502
table not found at 202506020603


NameError: name 'update_rawdf' is not defined

In [ ]:
#これでとれないのはほぼ障害レースか、前日のレースのみ

In [29]:
html_paths_race_prediction = [
    scraping_prediction.HTML_RACE_DIR / f"{race_id}.bin" for race_id in race_id_list_prediction
]

race_info_pre = scraping_prediction.create_race_info(html_path_list=html_paths_race_prediction)

  0%|          | 0/192 [00:00<?, ?it/s]